In [40]:
import os
import pandas as pd
from google.cloud import translate_v2 as translate
from google.oauth2 import service_account
import json
from googleapiclient.discovery import build
import urllib.parse


In [41]:

# Define the path to the file containing the project ID and API key
config_file = '/Users/aly.milne/Library/CloudStorage/OneDrive-BrighamYoungUniversity/Fall 2023/STAT 386/ST386_Final_Project/api-key.txt'

# Initialize variables
project_id1 = None
api_key1 = None

# Read the project ID and API key from the file
with open(config_file, 'r') as file:
    lines = file.readlines()  # Read all lines into a list

    # Extract project ID from the first line, if available
    if len(lines) > 0:
        project_id = lines[0].strip()

    # Extract API key from the second line, if available
    if len(lines) > 1:
        api_key = lines[1].strip()
        


In [42]:
# Initialize the translation client
translate_client = translate.Client()

# Cache for storing translations
translation_cache = {}

def batch_translate_text_with_model(target: str, texts: list, model: str = "nmt") -> list:
    translated_texts = []
    for text in texts:
        if text in translation_cache:
            # Use cached translation if available
            translated_texts.append(translation_cache[text])
        else:
            if isinstance(text, bytes):
                text = text.decode("utf-8")

            result = translate_client.translate(text, target_language=target, model=model)
            translated_text = result["translatedText"]
            translation_cache[text] = translated_text
            translated_texts.append(translated_text)
    return translated_texts


In [43]:

# Read the CSV file
df = pd.read_csv('Scraping_Code/trade_w_locations.csv', encoding='utf-8')

# Identify text columns
text_columns = df.select_dtypes(include=['object']).columns
# Remove "Unique Name" from the list
text_columns = [col for col in text_columns if col != "Unique Name"]
# Remove "Museum" from the list
text_columns = [col for col in text_columns if col != "Museum"]
# Remove "Catalog Link" from the list
text_columns = [col for col in text_columns if col != "Catalog Link"]


In [44]:
text_columns

['Föremålsbenämning',
 'Föremålsnr.',
 'Andra nummer',
 'Material',
 'Plats',
 'Fornlämning',
 'Socken',
 'Landskap',
 'Land',
 'Kontexttyp',
 'Kontextnr.',
 'Kategori',
 'Storlek',
 'Datering',
 'Tidsperiod',
 'Föremålsnummer',
 'Omnämns i katalog',
 'Fyndplats',
 'Arkeologisk kontext',
 'Förvärvsmetod',
 'Kontextnamn',
 'Historisk plats',
 'Undersökare',
 'Undersökningsår']

In [46]:

for column in text_columns:
    # Process the column in batches
    for i in range(0, len(df), batch_size):
        batch_slice = slice(i, i + batch_size)
        batch_texts = df[column][batch_slice].dropna()
        translated_batch = batch_translate_text_with_model('en', batch_texts.tolist())

        # Update only the rows that were translated
        df.loc[batch_texts.index, column + '_translated'] = translated_batch

In [47]:
df.to_csv('trade_translated.csv', index=False)